# stata_session

> A class for representing a Stata session

Some parts adapted from the [stata_kernel version](https://github.com/kylebarron/stata_kernel/blob/master/stata_kernel/completions.py), limited for now to variables, globals, locals, scalars, matrices, and file names.

In [ ]:
#| default_exp stata_session
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from nbstata.stata_more import diverted_stata_output_quicker, local_names, run_sfi
from nbstata.stata_more import get_local_dict as _get_local_dict
from nbstata.noecho import run_as_program_w_locals as _run_as_program_w_locals
from fastcore.basics import patch_to
from textwrap import dedent
import re

In [ ]:
from nbstata.stata import get_local

In [ ]:
#| export
class StataSession():
    def __init__(self):
        """"""
        self.matchall = re.compile(
            r"\A.*?"
            r"^%varlist%(?P<varlist>.*?)"
            r"%globals%(?P<globals>.*?)"
            #r"%locals%(?P<locals>.*?)"
            r"%scalars%(?P<scalars>.*?)"
            r"%matrices%(?P<matrices>.*?)%end%", #"(\Z|---+\s*end)",
            flags=re.DOTALL + re.MULTILINE).match

        # Varlist-style matching; applies to most
        self.varlist = re.compile(r"(?:\s+)(\S+)", flags=re.MULTILINE)

        # file-style matching
        self.filelist = re.compile(r"[\r\n]{1,2}", flags=re.MULTILINE)

        # Clean line-breaks.
        self.varclean = re.compile(
            r"(?=\s*)[\r\n]{1,2}?^>\s", flags=re.MULTILINE).sub
        
        #         # Match output from mata mata desc
#         self.matadesc = re.compile(
#             r"(\A.*?---+|---+[\r\n]*\Z)", flags=re.MULTILINE + re.DOTALL)

#         self.matalist = re.compile(
#             r"(?:.*?)\s(\S+)\s*$", flags=re.MULTILINE + re.DOTALL)

#         self.mataclean = re.compile(r"\W.*?(\b|$)")
#         self.matasearch = re.compile(r"(?P<kw>\w.*?(?=\W|\b|$))").search

        self.clear_suggestions()
#         self.suggestions = self.get_suggestions(kernel)
#         self.suggestions['magics'] = kernel.magics.available_magics
#         self.suggestions['magics_set'] = config.all_settings

    def clear_suggestions(self):
        self.suggestions = None

In [ ]:
#| export
@patch_to(StataSession)
def refresh_suggestions(self):
    self.suggestions = self.get_suggestions()
#     self.suggestions['magics_set'] = config.all_settings
#     self.globals = self.get_globals(kernel)

In [ ]:
#| eval: false
from nbstata.config import launch_stata

In [ ]:
#| export
@patch_to(StataSession)
def _completions(self):
#     return dedent(f"""\
#     %varlist%
#     {' '.join(variable_names())}
#     %globals%
#     {' '.join(global_names())}
#     """
    return diverted_stata_output_quicker(dedent("""\
        local _temp_completions_while_local_ = 1
        while `_temp_completions_while_local_' {
        set more off
        set trace off
        if `"`varlist'"' != "" {
        local _temp_completions_varlist_loc_ `"`varlist'"'
        }
        syntax [varlist]
        disp "%varlist%"
        disp `"`varlist'"'
        macro drop _varlist __temp_completions_while_local_
        if `"`_temp_completions_varlist_loc_'"' != "" {
        local varlist `"`_temp_completions_varlist_loc_'"'
        macro drop __temp_completions_varlist_loc_
        }
        disp "%globals%"
        disp `"`:all globals'"'
        *disp "%locals%"
        *mata : invtokens(st_dir("local", "macro", "*")')
        disp "%scalars%"
        disp `"`:all scalars'"'
        disp "%matrices%"
        disp `"`:all matrices'"'
        disp "%end%"
        local _temp_completions_while_local_ = 0
        }
        macro drop _temp_completions_while_local_
    """))

In [ ]:
show_doc(StataSession._completions)

---

[source](https://github.com/hugetim/nbstata/blob/main/nbstata/stata_session.py#L64){target="_blank" style="float:right; font-size:smaller"}

### StataSession._completions

>      StataSession._completions ()

In [ ]:
#| eval: false
launch_stata(splash=False)

In [ ]:
#| eval: False
run_sfi("""\
local varlist = 5
local varlist1 = 5""")

In [ ]:
#| hide
#| eval: False
test_instance = StataSession()
print(test_instance._completions())

%varlist%

%globals%
S_level F1 F2 F7 F8 S_ADO S_StataMP S_StataSE S_CONSOLE S_FLAVOR S_OS S_OSDTL S
> _MACH
%scalars%

%matrices%

%end%



In [ ]:
#| export
@patch_to(StataSession)
def _get_locals(self):
    return self.suggestions['locals'] if self.suggestions else local_names()

In [ ]:
from fastcore.test import test_eq

In [ ]:
#| hide
#| eval: False
test_eq(set(test_instance._get_locals()), {'varlist', 'varlist1'})

In [ ]:
#| export
@patch_to(StataSession)
def get_suggestions(self):
    match = self.matchall(self._completions())
    suggestions = match.groupdict()
#         suggestions['mata'] = self._parse_mata_desc(suggestions['mata'])
#         suggestions['programs'] = self._parse_programs_desc(
#             suggestions['programs'])
    for k, v in suggestions.items():
#             if k in ['mata', 'programs']:
#                 continue
#             elif k in ['logfiles']:
#                 suggestions[k] = [
#                     f for f in self.filelist.split(v.strip()) if f]
#             else:
        suggestions[k] = self.varlist.findall(self.varclean('', v))
    suggestions['locals'] = self._get_locals()
    return suggestions

In [ ]:
#| eval: false
test_instance.refresh_suggestions()
test_instance.suggestions

{'varlist': [],
 'globals': ['S_level',
  'F1',
  'F2',
  'F7',
  'F8',
  'S_ADO',
  'S_StataMP',
  'S_StataSE',
  'S_CONSOLE',
  'S_FLAVOR',
  'S_OS',
  'S_OSDTL',
  'S_MACH'],
 'scalars': [],
 'matrices': [],
 'locals': ['varlist1', 'varlist']}

In [ ]:
#| eval: false
run_sfi(dedent("""
    local local1 = 1
    local local2 "two"
    local local3 `""3""' """))
print(repr(get_local("local1")))
print(repr(get_local("local2")))
print(repr(get_local("local3")))

'1'
'two'
'"3"'


In [ ]:
#| export
@patch_to(StataSession)
def get_local_dict(self):
    return _get_local_dict(self._get_locals())

In [ ]:
#| eval: False
run_sfi('''\
macro drop _all
local test1 "blah blah" ''')
test_instance.clear_suggestions()
test_eq(test_instance.get_local_dict(), {'test1': 'blah blah'})
run_sfi('local test1 ""')

In [ ]:
#| export
@patch_to(StataSession)
def run_as_program_w_locals(self, std_code):
    """After `break_out_prog_blocks`, run noecho, inserting locals when needed"""
    return _run_as_program_w_locals(std_code, local_dict=self.get_local_dict())

In [ ]:
#| eval: false
run_sfi(dedent("""
    macro drop _all
    local local1 = 1
    local local2 "two"
    local local3 `""3""' """))
test_instance.clear_suggestions()
test_instance.run_as_program_w_locals("""disp `"`local1' `local2' `local3'"' """)

1 two "3"


In [ ]:
#| eval: false
code = '''\
local test1 "blah blah"
local test2 "blah"
'''
test_instance.clear_suggestions()
test_instance.run_as_program_w_locals("""disp `"`local1' `local2' `local3'"' \n""" + code)
test_eq(test_instance.get_local_dict(), 
        {'test2': 'blah',
         'test1': 'blah blah',
         'local1': '1',
         'local2': 'two',
         'local3': '"3"'})

1 two "3"


In [ ]:
#| eval: false
from nbstata.noecho import run_noecho, dispatch_run

In [ ]:
#| eval: false
test_instance.clear_suggestions()
run_noecho(dedent("""\
    disp `"`local1' `local2' `local3'"'
    disp `"`local1' `local2' `local3' `test1'"'
    """), run_as_prog=test_instance.run_as_program_w_locals)

1 two "3"
1 two "3" blah blah


In [ ]:
#| eval: false
code = """\
local local1 "foo"
local local2 "bar"
local abcd "foo bar"
"""
test_instance.clear_suggestions()
run_noecho(code, run_as_prog=test_instance.run_as_program_w_locals)
test_instance.clear_suggestions()
run_noecho(dedent("""\
    disp `"`local1' `local2' `local3'"'
    disp `"`local1' `local2' `local3' `test1'"'
    """), run_as_prog=test_instance.run_as_program_w_locals)

foo bar "3"
foo bar "3" blah blah


In [ ]:
#| eval: false
test_instance.clear_suggestions()
code2 = '''\
display "line continuation " /// commented out
    "comment"'''
dispatch_run(code2, noecho=True,
             run_as_prog=test_instance.run_as_program_w_locals)

line continuation comment


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()